## Loading the data

In [1]:
#"""
!wget https://s3-us-west-1.amazonaws.com/udacity-dlnfd/datasets/celeba.zip
import zipfile, os
with zipfile.ZipFile(os.getcwd() + '/celeba.zip', 'r') as zip_ref:
  zip_ref.extractall(os.getcwd()+'/data_faces')
#"""

--2020-06-27 13:03:33--  https://s3-us-west-1.amazonaws.com/udacity-dlnfd/datasets/celeba.zip
Resolving s3-us-west-1.amazonaws.com (s3-us-west-1.amazonaws.com)... 52.219.120.128
Connecting to s3-us-west-1.amazonaws.com (s3-us-west-1.amazonaws.com)|52.219.120.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1443490838 (1.3G) [application/zip]
Saving to: ‘celeba.zip’

celeba.zip          100%[===================>]   1.34G  48.3MB/s    in 27s     

2020-06-27 13:04:00 (51.7 MB/s) - ‘celeba.zip’ saved [1443490838/1443490838]



# Importing the libraries

In [2]:
import keras
import numpy as np
import matplotlib.pyplot as plt
import os

Using TensorFlow backend.


## Defining the GAN

In [3]:
def define_GAN(g_model, d_model):
    d_model.trainable = False
    model = keras.models.Sequential()
    model.add(g_model)
    model.add(d_model)
    opt = keras.optimizers.adam(learning_rate= 0.0002,
                                beta_1= 0.5)
    model.compile(loss= 'binary_crossentropy', optimizer= opt)
    return model

## Defining the Discriminator

In [4]:
def define_discriminator(input_shape= (96,80,3)):
    model = keras.models.Sequential()
    
    model.add(keras.layers.Conv2D(filters= 128,
                                  kernel_size= (3,3),
                                  padding= 'same',
                                  input_shape= input_shape))
    model.add(keras.layers.LeakyReLU(0.2))
    model.add(keras.layers.Dropout(0.4))

    # 96 * 80 * 3
    model.add(keras.layers.Conv2D(filters= 128,
                                  kernel_size= (3,3),
                                  strides= (2,2),
                                  padding= 'same'))
    model.add(keras.layers.LeakyReLU(0.2))
    model.add(keras.layers.Dropout(0.4))
    
    # 48 * 40 * 3
    model.add(keras.layers.Conv2D(filters= 128,
                                  kernel_size= (4,4),
                                  strides= (2,2),
                                  padding= 'same'))
    model.add(keras.layers.LeakyReLU(0.2))
    model.add(keras.layers.Dropout(0.4))
    
    # 24 * 20 * 3
    model.add(keras.layers.Conv2D(filters= 128,
                                  kernel_size= (4,4),
                                  strides= (2,2),
                                  padding= 'same'))
    model.add(keras.layers.LeakyReLU(0.2))
    model.add(keras.layers.Dropout(0.4))
    
    # 12 * 10 * 3

    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dropout(0.4))
    model.add(keras.layers.Dense(units= 1,
                                 activation= 'sigmoid'))
    opt = keras.optimizers.adam(learning_rate= 0.0002, beta_1= 0.5)
    model.compile(loss= 'binary_crossentropy', optimizer= opt, metrics= ['accuracy'])
    
    return model

## Defining the Generator

In [5]:
def define_generator(latent_dim):
    model = keras.models.Sequential()

    model.add(keras.layers.Dense(units= 256 * 6 * 5, input_dim= latent_dim))
    model.add(keras.layers.LeakyReLU(0.2))
    model.add(keras.layers.Reshape((6, 5, 256)))
    # 6 * 5
    model.add(keras.layers.Conv2DTranspose(filters= 256,
                                           kernel_size= (4,4),
                                           padding= 'same',
                                           strides= (2,2)))
    model.add(keras.layers.LeakyReLU(0.2))
    # 12 * 10
    model.add(keras.layers.Conv2DTranspose(filters= 128,
                                           kernel_size= (4,4),
                                           padding= 'same',
                                           strides= (2,2)))
    model.add(keras.layers.LeakyReLU(0.2))
    # 24 * 20
    model.add(keras.layers.Conv2DTranspose(filters= 128,
                                           kernel_size= (4,4),
                                           padding= 'same',
                                           strides= (2,2)))
    model.add(keras.layers.LeakyReLU(0.2))
    # 48 * 40
    model.add(keras.layers.Conv2DTranspose(filters= 128,
                                           kernel_size= (4,4),
                                           padding= 'same',
                                           strides= (2,2)))
    model.add(keras.layers.LeakyReLU(0.2))
    # 96 * 80
    
    # Ready for colorization
    model.add(keras.layers.Conv2D(filters= 3,
                                  kernel_size= (3,3),
                                  padding= 'same',
                                  activation= 'tanh'))
    
    return model

## Generating the Fake samples

In [6]:
def generate_latent_points(latent_dim, n_samples):
    X = np.random.randn(latent_dim * n_samples)
    X = X.reshape((n_samples, latent_dim))
    return X

In [7]:
def generate_fake_sample(g_model, latent_dim, n_samples):
    x_input = generate_latent_points(latent_dim= latent_dim,
                                     n_samples= n_samples)
    
    X = g_model.predict(x_input)
    y = np.zeros((n_samples, 1))
    return X, y

## Generating the Real samples

In [8]:
def generate_real_sample(train_it):
    X, _ = train_it.next()
    X = X[:, 6:102,4:84,:].astype('float32')
    X = (X - 127.5) / 127.5
    y = np.ones((X.shape[0], 1))
    # y = 0.9 * np.ones((n_samples, 1)) + 0.2 * np.random.rand(n_samples).reshape((n_samples, 1))
    return X, y

## Saving the Plot and summarizing

In [9]:
def save_plot(x_input, epoch, n=5):
    x_input = (x_input + 1.0) / 2.0
    filename = f'drive/My Drive/CGAN/generated_{epoch + 1}.png'
    for i in range(n*n):
        plt.subplot(n, n, i+1)
        plt.imshow(x_input[i,:,:,:])
        plt.axis('off')
    plt.savefig(filename)
    plt.close()

In [10]:
def summarize_the_model(g_model, d_model, epoch, latent_dim, n_samples, train_iter):
    X_real, y_real = generate_real_sample(train_it= train_iter)
    X_fake, y_fake = generate_fake_sample(g_model= g_model,
                                          latent_dim= latent_dim,
                                          n_samples= n_samples)
    print(f'Accuracy on real data: {d_model.evaluate(X_real, y_real, verbose= 0)}')
    print(f'Accuracy on fake data: {d_model.evaluate(X_fake, y_fake, verbose= 0)}')
    filename_g = f'drive/My Drive/CGAN/g_model_e_{epoch + 1:02}.h5'
    filename_d = f'drive/My Drive/CGAN/d_model_e_{epoch + 1:02}.h5'
    
    save_plot(x_input= X_fake,
              epoch= epoch)

    g_model.save(filename_g)
    d_model.save(filename_d)

# Function for training the GAN

In [11]:
def train_GAN(gan_model, g_model, d_model, dataset_len, latent_dim, train_iter, iters= 100, batch_size= 256, last_epoch= None):
    half_batch = int(batch_size / 2)
    batch_per_epoch = int(dataset_len / batch_size)
    for i in range(iters):
        for j in range(batch_per_epoch):
            X_real, y_real = generate_real_sample(train_it= train_it)
            X_fake, y_fake = generate_fake_sample(g_model= g_model,
                                                  latent_dim= latent_dim,
                                                  n_samples= half_batch)
            X, y = np.vstack((X_real, X_fake)), np.vstack((y_real, y_fake))
            dloss = d_model.train_on_batch(X, y)
            x_gan = generate_latent_points(latent_dim= latent_dim,
                                             n_samples= batch_size)
            y_gan = np.ones((batch_size, 1))
            gloss = gan_model.train_on_batch(x_gan, y_gan)
            print(f'> Epoch: {i+1}, {j+1}/{batch_per_epoch}, dloss: {dloss[0]:5f}, gloss: {gloss:5f}')
        # After each epoch
        epoch_number = 0
        if last_epoch is not None:
          epoch_number = i + int(last_epoch)
        else:
          epoch_number = i

        summarize_the_model(g_model= g_model,
                            d_model= d_model,
                            epoch= epoch_number,
                            latent_dim= latent_dim,
                            n_samples= batch_size,
                            train_iter= train_it)

In [12]:
def load_models(latent_dim, last_epoch= None):
  if last_epoch is not None:
    g_model = keras.models.load_model(f'drive/My Drive/CGAN/g_model_e_{last_epoch}.h5')
    d_model = keras.models.load_model(f'drive/My Drive/CGAN/d_model_e_{last_epoch}.h5')
    
    d_model.trainable = False
    gan_model = keras.models.Sequential()
    gan_model.add(g_model)
    gan_model.add(d_model)
    opt = keras.optimizers.adam(learning_rate= 0.0002,
                                beta_1= 0.5)
    gan_model.compile(loss= 'binary_crossentropy', optimizer= opt)

  else:
    g_model = define_generator(latent_dim= latent_dim)
    d_model = define_discriminator()
    gan_model = define_GAN(g_model=g_model,
                          d_model= d_model)
    
  return g_model, d_model, gan_model

## Training the Model

In [16]:
latent_dim = 200
batch_size = 128
last_epoch = "49"

g_model, d_model, gan_model = load_models(latent_dim,
                                          last_epoch= last_epoch)

datagen = keras.preprocessing.image.ImageDataGenerator()
path = os.getcwd() + '/data_faces'
train_it = datagen.flow_from_directory(path,
                                       batch_size= int(batch_size/2),
                                       target_size= (109, 89))

train_GAN(gan_model= gan_model,
          g_model= g_model,
          d_model= d_model,
          dataset_len= 202599,
          latent_dim= latent_dim,
          train_iter= train_it,
          last_epoch= last_epoch)

/usr/local/lib/python3.6/dist-packages/keras/engine/saving.py:341: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


Found 202599 images belonging to 1 classes.


/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


Streaming output truncated to the last 5000 lines.
> Epoch: 11, 643/791, dloss: 0.544701, gloss: 1.329196
> Epoch: 11, 644/791, dloss: 0.496082, gloss: 1.224070
> Epoch: 11, 645/791, dloss: 0.654148, gloss: 1.980623
> Epoch: 11, 646/791, dloss: 0.720265, gloss: 0.971625
> Epoch: 11, 647/791, dloss: 0.904051, gloss: 1.967930
> Epoch: 11, 648/791, dloss: 0.719920, gloss: 1.488960
> Epoch: 11, 649/791, dloss: 0.685681, gloss: 1.064464
> Epoch: 11, 650/791, dloss: 0.610535, gloss: 0.985487
> Epoch: 11, 651/791, dloss: 0.596171, gloss: 1.035070
> Epoch: 11, 652/791, dloss: 0.613754, gloss: 1.199324
> Epoch: 11, 653/791, dloss: 0.590676, gloss: 1.333433
> Epoch: 11, 654/791, dloss: 0.631106, gloss: 1.249714
> Epoch: 11, 655/791, dloss: 0.562788, gloss: 1.228330
> Epoch: 11, 656/791, dloss: 0.577265, gloss: 1.236357
> Epoch: 11, 657/791, dloss: 0.596369, gloss: 1.260369
> Epoch: 11, 658/791, dloss: 0.637062, gloss: 1.292771
> Epoch: 11, 659/791, dloss: 0.618009, gloss: 1.324688
> Epoch: 11, 6

KeyboardInterrupt: ignored

In [17]:
!nvidia-smi

Sat Jun 27 16:36:59 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.36.06    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   64C    P0    41W / 250W |  11877MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------